###                                              **OUTOLOADER**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys

proj_path=os.path.join(os.getcwd(),'..','..')
sys.path.append(proj_path)

from utils.transformations import transformations

### **DimUser**

In [0]:
df_user=spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemaLocation","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimUser/checkpoint")\
            .load("abfss://bronze@storagespotifyazure.dfs.core.windows.net/DimUser")


In [0]:
display(df_user)


In [0]:
df_user=df_user.withColumn("user_name",upper(col("user_name")))
display(df_user)

In [0]:
drop_col=transformations()
df_user=drop_col.dropCol(df_user,["_rescued_data"])
df_user=df_user.dropDuplicates(["user_id"])
display(df_user)

In [0]:
df_user.writeStream.format("delta")\
    .outputMode("append")\
        .option("checkpointLocation","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimUser/checkpoint")\
            .trigger(once=True)\
                .option("path","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimUser/data")\
                    .toTable("spotify_cata.silver.DimUser")


### **DimArtist**

In [0]:
df_art=spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","parquet")\
        .option("cloudFiles.schemaLocation","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimArtist/checkpoint")\
            .load("abfss://bronze@storagespotifyazure.dfs.core.windows.net/DimArtist")



In [0]:
display(df_art)

In [0]:
df_art=df_art.withColumn("artist_name",upper(col("artist_name")))
display(df_art)

In [0]:
drop_col=transformations()
df_art=drop_col.dropCol(df_art,["_rescued_data"])
df_art=df_art.dropDuplicates(["artist_id"])
display(df_art)

In [0]:
df_art.writeStream.format("delta")\
    .outputMode("append")\
        .option("checkpointLocation","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimArtist/checkpoint")\
            .trigger(once=True)\
                .option("path","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimArtist/data")\
                .toTable("spotify_cata.silver.DimArtist")

### **DimTrack**

In [0]:
df_track = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation",
            "abfss://silver@storagespotifyazure.dfs.core.windows.net/DimTrack/checkpoint")\
    .load("abfss://bronze@storagespotifyazure.dfs.core.windows.net/DimTrack")

In [0]:
df_track = df_track.withColumn("track_name", upper(col("track_name")))
display(df_track)

In [0]:
df_track=df_track.withColumn("durationFlag",when(col("duration_sec")<150,"Low")\
                                          .when(col("duration_sec")<300,"Medium")\
                                          .otherwise("High"))
df_track=df_track.withColumn("track_name",regexp_replace(col("track_name"),"-"," "))
display(df_track)

In [0]:
drop_col = transformations()
df_track = drop_col.dropCol(df_track, ["_rescued_data"])
df_track = df_track.dropDuplicates(["track_id"])
display(df_track)

In [0]:
df_track.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimTrack/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimTrack/data")\
    .toTable("spotify_cata.silver.DimTrack")

### **FactStream**

In [0]:
df_stream = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation",
            "abfss://silver@storagespotifyazure.dfs.core.windows.net/FactStream/checkpoint")\
    .load("abfss://bronze@storagespotifyazure.dfs.core.windows.net/FactStream")


In [0]:

display(df_stream)

In [0]:
drop_col = transformations()
df_stream = drop_col.dropCol(df_stream, ["_rescued_data"])
df_stream = df_stream.dropDuplicates(["stream_id"])

In [0]:
df_stream.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation","abfss://silver@storagespotifyazure.dfs.core.windows.net/FactStream/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@storagespotifyazure.dfs.core.windows.net/FactStream/data")\
    .toTable("spotify_cata.silver.FactStream")

### **DimDate**

In [0]:
df_date = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation",
            "abfss://silver@storagespotifyazure.dfs.core.windows.net/DimDate/checkpoint")\
    .load("abfss://bronze@storagespotifyazure.dfs.core.windows.net/DimDate")

In [0]:
display(df_date)

In [0]:
drop_col = transformations()
df_date = drop_col.dropCol(df_date, ["_rescued_data"])
display(df_date)

In [0]:
df_date.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@storagespotifyazure.dfs.core.windows.net/DimDate/checkpoint")\
    .trigger(once=True)\
    .option("path","abfss://silver@storagespotifyazure.dfs.core.windows.net/DimDate/data")\
    .toTable("spotify_cata.silver.DimDate")